In [1]:
import pandas as pd

ts = pd.read_csv("../data/processed/unsw_attack_timeseries.csv")
ts.head()


,time,attacks
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [2]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
ts['scaled'] = scaler.fit_transform(ts[['attacks']])


In [4]:
import numpy as np

sequence_length = 10

X = []
y = []

data = ts['scaled'].values

for i in range(len(data) - sequence_length):
    X.append(data[i:i+sequence_length])
    y.append(data[i+sequence_length])

X = np.array(X)
y = np.array(y)

X = X.reshape((X.shape[0], X.shape[1], 1))


In [7]:
import torch
import torch.nn as nn

class LSTMForecaster(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=32, num_layers=1, batch_first=True)
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out




In [8]:
model = LSTMForecaster()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

epochs = 30
for epoch in range(epochs):
    optimizer.zero_grad()
    output = model(X_tensor)
    loss = criterion(output, y_tensor)
    loss.backward()
    optimizer.step()

    if epoch % 5 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.5f}")


Epoch 0, Loss: 0.90169
Epoch 5, Loss: 0.80543
Epoch 10, Loss: 0.71442
Epoch 15, Loss: 0.62337
Epoch 20, Loss: 0.52717
Epoch 25, Loss: 0.42031


In [9]:
last_sequence = torch.tensor(data[-sequence_length:], dtype=torch.float32).view(1, sequence_length, 1)
next_scaled = model(last_sequence).item()
next_attack_count = scaler.inverse_transform([[next_scaled]])[0][0]

print("Predicted next attack count:", next_attack_count)


Predicted next attack count: 158.36966037750244


In [10]:
torch.save(model.state_dict(), "../models/lstm_forecast.pt")
import joblib
joblib.dump(scaler, "../models/attack_scaler.pkl")


['../models/attack_scaler.pkl']